In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain

from langchain.prompts import PromptTemplate

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory

from langchain.agents import load_tools

In [2]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
# llm = OpenAI(temperature=0, verbose=True)
llm = ChatOpenAI(model_name='gpt-4',temperature=0)
# llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613",temperature=0)

In [5]:
db = SQLDatabase.from_uri("sqlite:///./portfolio_data.db",
                         include_tables=['corporate_portfolio'],
                         sample_rows_in_table_info=2
)

In [6]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

sql_agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
) 

In [7]:
extra_tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [8]:
# from langchain.tools.sql_database.tool import (
#     InfoSQLDatabaseTool,
#     ListSQLDatabaseTool,
#     QuerySQLCheckerTool,
#     QuerySQLDataBaseTool,
# )

# sql_db_list_tables = ListSQLDatabaseTool(db=db, func=sql_agent.run)

# sql_db_schema = InfoSQLDatabaseTool(
#     db=db,
#     func=sql_agent.run,
#     description="""
#     Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. 
#     Be sure that the tables actually exist by calling sql_db_list_tables first! 
#     Example Input: 'table1, table2, table3'
#     """
# )

# sql_db_query = QuerySQLDataBaseTool(
#     db=db,
#     func=sql_agent.run,
#     description="""
#     Input to this tool is a detailed and correct SQL query, output is a result from the database. 
#     If the query is not correct, an error message will be returned. 
#     If an error is returned, rewrite the query, check the query, and try again. 
#     If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
#     """
# )

# sql_db_query_checker = QuerySQLCheckerTool(
#     llm=llm,
#     db=db,
#     func=sql_agent.run,
#     description="""
#     Use this tool to double check if your query is correct before executing it. 
#     Always use this tool before executing a query with sql_db_query!
#     """
# )

In [9]:
from langchain.tools import Tool, BaseTool
from pydantic.v1 import BaseModel, Field
import math
from scipy.stats import norm

def rw_corp(Input_String: str) -> str:

    """ 
    calculates the risk weight from input parameters 
    PD, LGD, MATURITY, F_LARGE_FIN and SIZE which are fed into the formula in this particular order 
    """
    
    PD_s, LGD_s, MATURITY_s, SIZE_s, F_LARGE_FIN_s = Input_String.split(",")

    PD = float(eval(PD_s.split(":")[1].strip()))
    LGD = float(eval(LGD_s.split(":")[1].strip()))
    MATURITY = float(eval(MATURITY_s.split(":")[1].strip()))
    SIZE = float(eval(SIZE_s.split(":")[1].strip()))
    F_LARGE_FIN = (F_LARGE_FIN_s.split(":")[1].strip())
       
    pd_final = max(0.0003, PD)
    size_final = max(5, min(SIZE, 50))    
    r0 = (0.12 * (1.0 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0))) + \
        (0.24 * (1.0 - (1 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0)))) - \
        (0.04 * (1.0 - (size_final - 5.0) / 45.0))
    if F_LARGE_FIN == 'Y':
        r = r0 * 1.25
    else:
        r = r0
    b = (0.11852 - 0.05478 * math.log(pd_final)) ** 2
    ma = ((1 - 1.5 * b) ** -1) * (1 + (MATURITY - 2.5) * b)    
    rw = ((LGD * norm.cdf((1 - r) ** -0.5 * norm.ppf(pd_final) + (r / (1 - r)) ** 0.5 * norm.ppf(0.999)) - pd_final * LGD) * ma) * 12.5 * 1.06
    return rw  


class RWInput(BaseModel):
    Input_String: str = Field(description='This is a string that contains values for the input parameters \
                              PD, LGD, MATURITY, SIZE and F_LARGE_FIN which are fed into the formula in this particular order ')
    
RWTool = Tool.from_function(
    func=rw_corp,
    name="RWTool",
    description="""
    This is a custom tool that calculates the risk weight from a set of input parameters:
        PD - Probability of Default,
        LGD - Loss Given Default,
        MATURITY - Remaining maturity of the loan in years,
        SIZE - The size of the client in MEUR, usually this is the client's turnover, 
        F_LARGE_FIN - If 'Y' the client is a Large Financial Institution        
        """,
    args_schema=RWInput
)

In [10]:
# tools.append(sql_db_query)
# tools.append(sql_db_schema)
# tools.append(sql_db_list_tables)
# tools.append(sql_db_query_checker)
extra_tools.append(RWTool)

In [11]:
tools = toolkit.get_tools() + list(extra_tools)

In [23]:
template = """

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: 'table1, table2, table3'
sql_db_list_tables: Input is an empty string, output is a comma separated list of tables in the database.
sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!
Calculator: Useful for when you need to answer questions about math.
RWTool: 
    This is a custom tool that calculates the risk weight from a set of input parameters:
        PD - Probability of Default,
        LGD - Loss Given Default,
        MATURITY - Remaining maturity of the loan in years,
        SIZE - The size of the client in MEUR, usually this is the client's turnover, 
        F_LARGE_FIN - If 'Y' the client is a Large Financial Institution        
        

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker, Calculator, RWTool]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history:
{history}

Question: {input}
Thought: I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables.
{agent_scratchpad}

"""

In [24]:
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    # tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # # Create a tools variable from the list of tools provided
        # kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # # Create a list of tool names for the tools provided
        # kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [25]:
prompt = CustomPromptTemplate(
    template=template,
    # tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

# from langchain import PromptTemplate

# prompt = PromptTemplate(
#     input_variables=["input", "intermediate_steps"],
#     template=template
# )

In [26]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [27]:
output_parser = CustomOutputParser()

In [28]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [29]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [30]:
from langchain.memory import ConversationBufferWindowMemory
memory=ConversationBufferWindowMemory(k=4)

In [31]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [32]:
agent_executor.run("What are total RWA and EAD for reporting date 31-12-2022?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""

Observation:corporate_portfolio
The corporate_portfolio table seems to be the only table in the database. I should check its schema to see if it contains the information I need.
Action: sql_db_schema
Action Input: "corporate_portfolio"

Observation:
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.35378	0.0058159	0.0	

'The total EAD for reporting date 31-12-2022 is 156,657,332.35 and the total RWA is 85,075,812.12.'

In [21]:
agent_executor.run("What is the change in total RWA from reporting date 2022-12-31 to reporting date 2023-03-31?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""

Observation:corporate_portfolio
The corporate_portfolio table seems to be the most relevant one for this question. I should check its schema to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "corporate_portfolio"

Observation:
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.353

'The change in total RWA from reporting date 2022-12-31 to reporting date 2023-03-31 is -7928980.59.'

In [22]:
agent_executor.run("What is the total exposure of only those loan ids that have status from Performing on reporting date 2022-12-31 and status Non Performing on reporting date 2023-03-31?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""

Observation:corporate_portfolio
The corporate_portfolio table seems to be the most relevant one for this query. I should check its schema to understand the structure and the fields it contains.
Action: sql_db_schema
Action Input: "corporate_portfolio"

Observation:
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.3537

'The total exposure of only those loan ids that have status from Performing on reporting date 2022-12-31 and status Non Performing on reporting date 2023-03-31 is 589368.68.'

#### sql + rw

In [33]:
agent_executor.run("Read the information in table corporate_portfolio for loan id 1439964799kgeq68 and for reporting date 31-12-2022")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""

Observation:corporate_portfolio
The corporate_portfolio table is available in the database. I should now check the schema of this table to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "corporate_portfolio" 

Observation:
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.35378	0

'The information for loan id 1439964799kgeq68 and for reporting date 31-12-2022 is as follows: \n- Reporting Date: 2022-12-31\n- Partner ID: dninx621xv9731\n- Loan ID: 1439964799kgeq68\n- Product: CREDIT_CARD\n- Status: Performing\n- Exposure Drawn: 5141.569897\n- Exposure Undrawn: 4578.250143\n- Exposure: 9719.82004\n- EAD: 7061.256068\n- EAD Coll: 0.0\n- PD: 0.0156611\n- LGD: 0.7026\n- Size: 2.345659392\n- Maturity: 1.41369863\n- F Large Fin: N\n- RW: 1.196209223\n- RWA: 8446.739634'

In [34]:
agent_executor.run("What will be the RW if PD doubles")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""

Observation:corporate_portfolio
The corporate_portfolio table seems to be the most relevant one. I should check its schema to confirm.
Action: sql_db_schema
Action Input: "corporate_portfolio"

Observation:
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.35378	0.0058159	0.0	0.123770567	3.63013699	N	0.0	0.0
2022-12-31

'If the Probability of Default (PD) doubles, the Risk Weight (RW) will be approximately 1.4635.'

In [35]:
agent_executor.run("What about if LGD doubles")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""

Observation:corporate_portfolio
The corporate_portfolio table seems to be the most relevant one for this question. I should query its schema to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "corporate_portfolio" 

Observation:
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.35

'If the Loss Given Default (LGD) doubles, the Risk Weight (RW) will be approximately 2.3924.'

In [36]:
agent_executor.run("And if MATURITY becomes 6.23")



> Entering new AgentExecutor chain...
Action: RWTool
Action Input: {"PD": 0.0156611, "LGD": 0.7026, "MATURITY": 6.23, "SIZE": 2.345659392, "F_LARGE_FIN": "N"}

Observation:1.9901793310985323
I now know the final answer.
Final Answer: If the remaining maturity of the loan becomes 6.23 years, the Risk Weight (RW) will be approximately 1.9902.

> Finished chain.


'If the remaining maturity of the loan becomes 6.23 years, the Risk Weight (RW) will be approximately 1.9902.'

In [37]:
agent_executor.run("What if all three parameters change like that?")



> Entering new AgentExecutor chain...
Action: RWTool
Action Input: {"PD": 0.0313222, "LGD": 1.4052, "MATURITY": 6.23, "SIZE": 2.345659392, "F_LARGE_FIN": "N"}

Observation:4.420645788063997
I now know the final answer.
Final Answer: If all three parameters change like that, the Risk Weight (RW) will be approximately 4.4206.

> Finished chain.


'If all three parameters change like that, the Risk Weight (RW) will be approximately 4.4206.'

In [38]:
agent_executor.run("Start over again and read the all information in table corporate_portfolio for loan id 1439964799kgeq68 but for reporting date 2023-03-31")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""

Observation:corporate_portfolio
The corporate_portfolio table is available. Now I need to check the schema of this table to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "corporate_portfolio"

Observation:
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.35378	0.0058159	0.0	0.1

'The information for loan id "1439964799kgeq68" and reporting date "2023-03-31" is as follows: \n- Reporting Date: 2023-03-31\n- Partner ID: dninx621xv9731\n- Loan ID: 1439964799kgeq68\n- Product: CREDIT_CARD\n- Status: Performing\n- Exposure Drawn: 4419.698264\n- Exposure Undrawn: 5181.260533\n- Exposure: 9600.958796\n- EAD: 6582.517105\n- EAD Coll: 0.0\n- PD: 0.0470619\n- LGD: 0.7026\n- Size: 2.345659392\n- Maturity: 1.16438356\n- F Large Fin: N\n- RW: 1.623828837\n- RWA: 10688.8811'

In [39]:
agent_executor.run("What is the change in PD for loan id 1439964799kgeq68 between reporting date 2022-12-31 and 2023-03-31")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""

Observation:corporate_portfolio
The corporate_portfolio table seems to be the most relevant one for this query. I should check its schema to confirm.
Action: sql_db_schema
Action Input: "corporate_portfolio" 

Observation:
CREATE TABLE corporate_portfolio (
	"REPORTING_DATE" TEXT, 
	"PARTNER_ID" TEXT, 
	"LOAN_ID" TEXT, 
	"PRODUCT" TEXT, 
	"STATUS" TEXT, 
	"EXPOSURE_DRAWN" REAL, 
	"EXPOSURE_UNDRAWN" REAL, 
	"EXPOSURE" REAL, 
	"EAD" REAL, 
	"EAD_COLL" REAL, 
	"PD" REAL, 
	"LGD" REAL, 
	"SIZE" REAL, 
	"MATURITY" REAL, 
	"F_LARGE_FIN" TEXT, 
	"RW" REAL, 
	"RWA" REAL
)

/*
2 rows from corporate_portfolio table:
REPORTING_DATE	PARTNER_ID	LOAN_ID	PRODUCT	STATUS	EXPOSURE_DRAWN	EXPOSURE_UNDRAWN	EXPOSURE	EAD	EAD_COLL	PD	LGD	SIZE	MATURITY	F_LARGE_FIN	RW	RWA
2022-12-31	uawif424nf9573	2507309957ifvm41	INVESTMENT	Performing	25088.77133	0.0	25088.77133	25131.35378	25131.35378	0.0058159	0.0	0.123770567	3.63013699	N	0.

'The change in PD for loan id 1439964799kgeq68 between reporting date 2022-12-31 and 2023-03-31 is approximately 0.0314.'